In [13]:
import os

root_path = "/home/choi/BrainDecoder/"
dir_path = os.path.join(root_path, "outputs", "samplelevel2img-samples", "samples")
dir_path

'/home/choi/BrainDecoder/outputs/samplelevel2img-samples/samples'

# SSIM

In [3]:
# https://gaussian37.github.io/vision-concept-ssim/
# https://scikit-image.org/docs/dev/api/skimage.metrics.html#skimage.metrics.structural_similarity

import cv2
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim

In [4]:
origin_name = "0069_00353"
img_name1 = "0069_00354"
img_name2 = "0069_00355"
img_name3 = "0069_00356"


def get_img_path(filename):
    if not filename.endswith(".png"):
        filename += ".png"
    return os.path.join(dir_path, filename)


origin = cv2.cvtColor(cv2.imread(get_img_path(origin_name)), cv2.COLOR_BGR2RGB)
img1 = cv2.cvtColor(cv2.imread(get_img_path(img_name1)), cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread(get_img_path(img_name2)), cv2.COLOR_BGR2RGB)
img3 = cv2.cvtColor(cv2.imread(get_img_path(img_name3)), cv2.COLOR_BGR2RGB)

ssim_1 = ssim(origin, img1, channel_axis=2)
# ssim_1, diff1 = ssim(origin, img1, channel_axis=2, full=True)
# diff1 = (diff1 * 255).astype("uint8")
ssim_2 = ssim(origin, img2, channel_axis=2)
# ssim_2, diff2 = ssim(origin, img2, channel_axis=2, full=True)
# diff2 = (diff2 * 255).astype("uint8")
ssim_3 = ssim(origin, img3, channel_axis=2)
# ssim_3, diff3 = ssim(origin, img3, channel_axis=2, full=True)
# diff3 = (diff3 * 255).astype("uint8")

print(ssim_1, ssim_2, ssim_3)

# 로컬 영역에서 SSIM 구하기
# ssim_1, diff1 = ssim(origin, img1, channel_axis=2, win_size=11, full=True)
# diff1 = (diff1 * 255).astype("uint8")

# 이미지간 차이 시각화하기
# fig, ax = plt.subplots(ncols=3)
# ax[0].imshow(diff1)
# ax[1].imshow(diff2)
# ax[2].imshow(diff3)
# plt.show()

0.11288692965251086 0.07803935657510917 0.09784093551992962


# Inception Score

In [14]:
import torch
from torchmetrics.image.inception import InceptionScore
from torchvision.transforms import transforms
from PIL import Image
# _ = torch.manual_seed(123)

In [24]:
gen_img1 = "0069_00354"
gen_img2 = "0069_00355"
gen_img3 = "0069_00356"

def get_img_path(filename):
    if not filename.endswith(".png"):
        filename += ".png"
    return os.path.join(dir_path, filename)

def get_img_tensor(filename):
    img = Image.open(get_img_path(filename))
    convert_tensor = transforms.ToTensor()
    return convert_tensor(img).type(dtype=torch.uint8)

# gen_imgs = [gen_img1,gen_img2,gen_img3]
# gen_imgs = torch.stack([get_img_tensor(filename) for filename in gen_imgs])
gen_imgs = torch.stack([get_img_tensor(filename) for filename in os.listdir(dir_path)])

In [25]:
inception = InceptionScore()
# generate some images
# imgs = torch.randint(0, 255, (100, 3, 299, 299), dtype=torch.uint8)
inception.update(gen_imgs)
inception.compute()

/home/choi/Downloads/miniconda3/envs/braindecoder/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


(tensor(1.0175), tensor(0.0117))

# FID

In [4]:
import torch
from torchmetrics.image.fid import FrechetInceptionDistance
import os
import numpy as np
from torchvision.transforms import transforms
from PIL import Image
# _ = torch.manual_seed(123)

In [9]:
gt_img1 = "0069_00353"
gt_img2 = "0069_00353"
gt_img3 = "0069_00353"
gen_img1 = "0069_00354"
gen_img2 = "0069_00355"
gen_img3 = "0069_00356"

def get_img_path(filename):
    if not filename.endswith(".png"):
        filename += ".png"
    return os.path.join(dir_path, filename)

def get_img_tensor(filename):
    img = Image.open(get_img_path(filename))
    convert_tensor = transforms.ToTensor()
    return convert_tensor(img).type(dtype=torch.uint8)

gt_imgs = [gt_img1,gt_img2,gt_img3]
gt_imgs = torch.stack([get_img_tensor(filename) for filename in gt_imgs])

gen_imgs = [gen_img1,gen_img2,gen_img3]
gen_imgs = torch.stack([get_img_tensor(filename) for filename in gen_imgs])

In [10]:
# feature = one of [64, 192, 768, 2048]
fid = FrechetInceptionDistance(feature=64)
# generate two slightly overlapping image intensity distributions
# imgs_dist1 = torch.randint(0, 200, (100, 3, 299, 299), dtype=torch.uint8)
# imgs_dist2 = torch.randint(100, 255, (100, 3, 299, 299), dtype=torch.uint8)
fid.update(gt_imgs, real=True)
fid.update(gen_imgs, real=False)
fid.compute()

/home/choi/Downloads/miniconda3/envs/braindecoder/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `FrechetInceptionDistance` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/choi/Downloads/miniconda3/envs/braindecoder/lib/python3.11/site-packages/scipy/linalg/_matfuncs_sqrtm.py:205: RuntimeWarning: invalid value encountered in scalar divide
  arg2 = norm(X.dot(X) - A, 'fro')**2 / norm(A, 'fro')


tensor(5.7531e-05)

# N-way Top-k Classification

In [11]:
import numpy as np
from torchmetrics.functional import accuracy
from torchvision.models import ViT_H_14_Weights, vit_h_14
import torch
from PIL import Image


In [45]:
@torch.no_grad()
def n_way_top_k_acc(pred, class_id, n_way, num_trials=40, top_k=1):
    pick_range = [i for i in np.arange(len(pred)) if i != class_id]
    acc_list = []
    for t in range(num_trials):
        idxs_picked = np.random.choice(pick_range, n_way - 1, replace=False)
        pred_picked = torch.cat([pred[class_id].unsqueeze(0), pred[idxs_picked]])
        acc = accuracy(
            pred_picked.unsqueeze(0), torch.tensor([0], device=pred.device), top_k=top_k
        )
        acc_list.append(acc.item())
    return np.mean(acc_list), np.std(acc_list)

In [55]:
@torch.no_grad()
def get_n_way_top_k_acc(
    pred_imgs, ground_truth, n_way, num_trials, top_k, device, return_std=False
):
    weights = ViT_H_14_Weights.DEFAULT
    model = vit_h_14(weights=weights)
    preprocess = weights.transforms()
    model = model.to(device)
    model = model.eval()

    acc_list = []
    std_list = []
    for pred, gt in zip(pred_imgs, ground_truth):
        pred = (
            preprocess(Image.fromarray(pred.astype(np.uint8))).unsqueeze(0).to(device)
        )
        gt = preprocess(Image.fromarray(gt.astype(np.uint8))).unsqueeze(0).to(device)
        gt_class_id = model(gt).squeeze(0).softmax(0).argmax().item()
        pred_out = model(pred).squeeze(0).softmax(0).detach()
        # print(pred_out.shape)
        # print(pred_out)
        # print(gt_class_id)
        # print(pred_out[gt_class_id])

        acc, std = n_way_top_k_acc(pred_out, gt_class_id, n_way, num_trials, top_k)
        acc_list.append(acc)
        std_list.append(std)

    if return_std:
        return acc_list, std_list
    return acc_list

In [56]:

origin_name = "0069_00353"
img_name1 = "0069_00354"
img_name2 = "0069_00355"
img_name3 = "0069_00356"
img_name4 = "0069_00349"

def get_img_path(filename):
    if not filename.endswith(".png"):
        filename += ".png"
    return os.path.join(dir_path, filename)

gt = Image.open(get_img_path(origin_name))
# gt = [np.array(gt),np.array(gt),np.array(gt)]
# display(gt)
gt = [np.array(gt)]

# canoe
pred1 = Image.open(get_img_path(img_name1))
pred2 = Image.open(get_img_path(img_name2))
pred3 = Image.open(get_img_path(img_name3))
# piano
pred4 = Image.open(get_img_path(img_name4))
# display(pred4)
# pred = [np.array(pred1),np.array(pred2),np.array(pred3)]
pred = [np.array(pred4)]

get_n_way_top_k_acc(
            pred,
            gt,
            n_way=50,
            num_trials=1000,
            top_k=1,
            device="cuda",
        )

[0.0]